## Libraries and variables

In [23]:
import h2o
from h2o.automl import H2OAutoML
import pandas as pd

data_path = "../01_Data/"
file_name_class = "4-1_psp-data_feateng_all_class.csv"
# file_name_class = "4-1_psp-data_feateng_featsel_class.csv"
# file_name_class = "4-1_psp-data_feateng_pcb_class.csv"

file_path_class = data_path + file_name_class

## Data load

In [24]:
df = pd.read_csv(file_path_class, index_col=0)
df

,success,amount_sc,3D_secured,transaction_hour_sc,card_Diners,card_Master,card_Visa,PSP_Goldcard,PSP_Moneycard,PSP_Simplecard,PSP_UK_Card
0,0,0.133013,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,1,0.133013,0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2,0,0.371795,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1,0.371795,1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0,0.189103,0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
50324,0,0.655449,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
50325,0,0.136218,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
50326,0,0.136218,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
50327,0,0.136218,0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0


## AutoML - Find baseline model

In [25]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
; OpenJDK 64-Bit Server VM (build 22+36-2370, mixed mode, sharing)
  Starting server from C:\Program Files\Python312\Lib\site-packages\h2o\backend\bin\h2o.jar
  Ice root: C:\Users\boehm\AppData\Local\Temp\tmp1i6w8jjx
  JVM stdout: C:\Users\boehm\AppData\Local\Temp\tmp1i6w8jjx\h2o_boehm_started_from_python.out
  JVM stderr: C:\Users\boehm\AppData\Local\Temp\tmp1i6w8jjx\h2o_boehm_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Berlin
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.2
H2O_cluster_version_age:,1 month and 16 days
H2O_cluster_name:,H2O_from_python_boehm_98agt2
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.944 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [26]:
psp_df = h2o.import_file(file_path_class)

psp_train, psp_test, psp_valid = psp_df.split_frame(ratios=[.7, .15])

y = "success"
x = psp_df.columns
x.remove(y)
x.remove("C1")

psp_train[y] = psp_train[y].asfactor()
psp_test[y] = psp_test[y].asfactor()

aml = H2OAutoML(max_models=20, seed =10, exclude_algos=["StackedEnsemble", "DeepLearning"], verbosity="info", nfolds=0)

aml.train(x=x, y=y, training_frame=psp_train, validation_frame=psp_valid)


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |
14:37:45.391: Project: AutoML_1_20240630_143745
14:37:45.392: Cross-validation disabled by user: no fold column nor nfolds > 1.
14:37:45.393: Setting stopping tolerance adaptively based on the training frame: 0.005327143237881408
14:37:45.394: Build control seed: 10
14:37:45.395: training frame: Frame key: AutoML_1_20240630_143745_training_py_25_sid_92b8    cols: 12    rows: 35238  chunks: 32    size: 523702  checksum: 70510806270666656
14:37:45.395: validation frame: Frame key: py_24_sid_92b8    cols: 12    rows: 7524  chunks: 32    size: 132438  checksum: 2934216017274820704
14:37:45.395: leaderboard frame: Frame key: py_24_sid_92b8    cols: 12    rows: 7524  chunks: 32    size: 132438  checksum: 2934216017274820704
14:37:45.395: blending frame: NULL
14:37:45.395: response column: success
14:37:45.396: fold column: null
14:37:45.396: weights column: null
14:37:45.407: Aut

Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_grid_1_AutoML_1_20240630_143745_model_7


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    35                 35                          54760                  7            13           10.6571       1             201           119.8

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.1454265623205273
RMSE: 0.38134834773541015
LogLoss: 0.4558882755896953
Mean Per-Class Error: 0.3430397147090495
AUC: 0.7227093981236932
AUCPR: 0.40385016176550803
Gini: 0.44541879624738634

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.2331546904153351
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      19894  8216   0.2923   (8216.0/28110.0)
1      2807   4321   0.3938   (2807.0/7128.0)
Total  22701  12537  0.3128   (11023.0/35238.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.233155     0.439461  215
max f2                       0.140585     0.602755  311
max f0point5                 0.282359     0.418902  162
max accuracy                 0.414819     0.804047  72
max precision                0.675762     1         0
max recall                   0.0582019    1         399
max specificity              0.675762     1         0
max absolute_mcc             0.271078     0.267669  174
max min_per_class_accuracy   0.219761     0.656706  231
max mean_per_class_accuracy  0.207937     0.659911  243
max tns                      0.675762     28110     0
max fns                      0.675762     7127      0
max fps                      0.0582019    28110     399
max tps                      0.0582019    7128      399
max tnr                      0.675762     1         0
max fnr                      0.675762     0.99986   0
max fpr                      0.0582019    1         399
max tpr                      0.0582019    1         399

Gains/Lift Table: Avg response rate: 20,23 %, avg score: 20,26 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100176                   0.522946           3.36109   3.36109            0.679887         0.584011   0.679887                    0.584011            0.03367         0.03367                    236.109   236.109            0.0296501
2        0.0200636                   0.456165           2.82093   3.09063            0.570621         0.482583   0.625177                    0.533225            0.0283389       0.062009                   182.093   209.063            0.0525817
3        0.0300244                   0.419049           2.63377   2.93906            0.532764         0.43638    0.594518                    0.501096            0.0262346       0.0882435                  163.377   193.906            0.0729821
4        0.0400136                   0.393559           2.3454    2.79086            0.474432         0.406522   0.564539                    0.477486            0.0234287       0.111672                   134.54    179.086            0.0898295
5        0.0500028                   0.370518           2.20496   2.67381            0.446023         0.381839   0.

In [27]:
lb = aml.leaderboard

In [28]:
print(lb)

model_id                                           auc    logloss     aucpr    mean_per_class_error      rmse       mse
GBM_grid_1_AutoML_1_20240630_143745_model_7   0.66141    0.482548  0.341643                0.368544  0.393618  0.154935
GBM_5_AutoML_1_20240630_143745                0.659728   0.482986  0.336491                0.372836  0.393841  0.155111
GBM_grid_1_AutoML_1_20240630_143745_model_12  0.659319   0.483447  0.338753                0.373769  0.394005  0.15524
GBM_1_AutoML_1_20240630_143745                0.658053   0.484131  0.340749                0.375097  0.394186  0.155382
GBM_2_AutoML_1_20240630_143745                0.657957   0.483755  0.336792                0.375823  0.39415   0.155354
GBM_grid_1_AutoML_1_20240630_143745_model_6   0.65705    0.484116  0.340203                0.373979  0.394269  0.155448
GBM_grid_1_AutoML_1_20240630_143745_model_5   0.656047   0.484818  0.333213                0.375793  0.394651  0.155749
GBM_3_AutoML_1_20240630_143745           

In [29]:
psp_pred = aml.leader.predict(psp_test)
psp_pred.head()

gbm prediction progress: |

███████████████████████████████████████████████████████| (done) 100%


predict,p0,p1
0,0.870661,0.129339
1,0.798436,0.201564
0,0.864662,0.135338
0,0.855505,0.144495
0,0.865547,0.134453
0,0.818441,0.181559
0,0.841214,0.158786
0,0.853515,0.146485
0,0.853515,0.146485
0,0.878794,0.121206


In [30]:
aml.leader.model_performance(psp_test)

ModelMetricsBinomial: gbm
** Reported on test data. **

MSE: 0.15196177336603856
RMSE: 0.3898227460860109
LogLoss: 0.47589848178551847
Mean Per-Class Error: 0.3740242471190516
AUC: 0.66215930150818
AUCPR: 0.3422576682989064
Gini: 0.32431860301636006

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.19887455741638052
       0     1     Error    Rate
-----  ----  ----  -------  ---------------
0      3522  2509  0.416    (2509.0/6031.0)
1      510   1026  0.332    (510.0/1536.0)
Total  4032  3535  0.399    (3019.0/7567.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.198875     0.404654  245
max f2                       0.116574     0.569521  340
max f0point5                 0.273033     0.342742  161
max accuracy                 0.460928     0.801242  47
max precision                0.683421     1         0
max recall                   0.0659303    1         397
max specificity              0.683421     1         0
max absolute_mcc             0.198875     0.20312   245
max min_per_class_accuracy   0.210994     0.622616  232
max mean_per_class_accuracy  0.198875     0.625976  245
max tns                      0.683421     6031      0
max fns                      0.683421     1535      0
max fps                      0.0591481    6031      399
max tps                      0.0659303    1536      397
max tnr                      0.683421     1         0
max fnr                      0.683421     0.999349  0
max fpr                      0.0591481    1         399
max tpr                      0.0659303    1         397

Gains/Lift Table: Avg response rate: 20,30 %, avg score: 20,04 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score      cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ---------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100436                   0.511495           2.98179   2.98179            0.605263         0.579882   0.605263                    0.579882            0.0299479       0.0299479                  198.179   198.179            0.0249736
2        0.0200872                   0.450281           2.7225    2.85215            0.552632         0.473103   0.578947                    0.526492            0.0273438       0.0572917                  172.25    185.215            0.0466798
3        0.0299987                   0.415475           2.49606   2.7345             0.506667         0.431248   0.555066                    0.495024            0.0247396       0.0820312                  149.606   173.45             0.0652844
4        0.0400423                   0.388047           2.13911   2.58516            0.434211         0.404053   0.524752                    0.472206            0.0214844       0.103516                   113.911   158.516            0.079639
5        0.0500859                   0.364233           1.55572   2.37873            0.315789         0.37606    0.48285                     0.452926            0.015625        0.119141                   55.5715   137.873            0.0866419
6        0.10004                     0.309858           1.51182   1.94584            0.306878         0.331573   0.39498                     0.39233             0.0755208       0.194661                   51.1815   94.5843            0.11872
7        0.149993                    0.287847           1.47272   1.78827            0.298942         0.297865   0.362996                    0.360869            0.0735677       0.268229                   47.2717   78.8273            0.148349
8        0.200079        

In [31]:
h2o.shutdown(prompt=False)

H2O session _sid_92b8 closed.


C:\Users\boehm\AppData\Local\Temp\ipykernel_2216\1954269801.py:1: H2ODeprecationWarning: Deprecated, use ``h2o.cluster().shutdown()``.
  h2o.shutdown(prompt=False)
